Script for Calculate PESQ

In [34]:
import os
import pandas as pd
import soundfile as sf
import librosa
from pesq import pesq, PesqError

# Define directories
filtered_dir = "/media/lab2208/New Volume/Organized Voice data HDD/WaveUNet/VPQAD_filtered"
nonfiltered_dir = "/media/lab2208/New Volume/Organized Voice data HDD/VPQAD_Cafeteria_raw_TD"

destination_path = "/media/lab2208/New Volume/Organized Voice data HDD/CMGAN/Results/PESQ/PESQ_WaveUNet_VPQAD.xlsx"

# Initialize list for results
results = []

# Iterate over files in the nonfiltered directory
for file_name in os.listdir(nonfiltered_dir):
    if file_name.endswith(".wav"):
        # Construct paths for nonfiltered and filtered files
        nonfiltered_path = os.path.join(nonfiltered_dir, file_name)
        filtered_path = os.path.join(filtered_dir, file_name)

        # Check if the corresponding filtered file exists
        if os.path.exists(filtered_path):
            # Load the nonfiltered and filtered audio files
            nonfiltered_audio, sr1 = sf.read(nonfiltered_path)
            filtered_audio, sr2 = sf.read(filtered_path)

            # Resample if the sampling rate is not 16000 Hz
            if sr1 != 16000:
                nonfiltered_audio = librosa.resample(nonfiltered_audio.T, orig_sr=sr1, target_sr=16000).T
                sr1 = 16000

            if sr2 != 16000:
                filtered_audio = librosa.resample(filtered_audio.T, orig_sr=sr2, target_sr=16000).T
                sr2 = 16000

            # Ensure both have the same sampling rate (should be 16000 Hz)
            if sr1 == sr2 == 16000:
                try:
                    # Ensure the audio arrays are 1D
                    if nonfiltered_audio.ndim > 1:
                        nonfiltered_audio = nonfiltered_audio[:, 0]
                    if filtered_audio.ndim > 1:
                        filtered_audio = filtered_audio[:, 0]

                    # Calculate PESQ
                    pesq_value = pesq(sr1, nonfiltered_audio, filtered_audio, 'nb')

                    # Append the results
                    results.append({
                        "File Name": file_name,
                        "PESQ Value": pesq_value
                    })
                except PesqError as e:
                    print(f"PESQ calculation failed for {file_name}: {e}")

# Convert results to a DataFrame and save to Excel
df = pd.DataFrame(results)
df.to_excel(destination_path, index=False)

print(f"PESQ evaluation completed. Results saved to {destination_path}")


PESQ evaluation completed. Results saved to /media/lab2208/New Volume/Organized Voice data HDD/CMGAN/Results/PESQ/PESQ_WaveUNet_VPQAD.xlsx
